In [75]:
import zipfile
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import ticker
import kaggle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import featuretools as ft
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelBinarizer, MultiLabelBinarizer
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import NearMiss
from numpy import where


import os
import composeml as cp
import featuretools as ft
import dask.dataframe as dd
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, recall_score
from sklearn.model_selection import GridSearchCV, validation_curve
from sklearn.metrics import plot_confusion_matrix
import seaborn as sns
from dask.distributed import Client

### Importing and preparing the data

In [8]:
#Convert csv to DataFrame
oct19 = pd.read_csv("../data/2019-Oct.csv")
nov19 = pd.read_csv("../data/2019-Nov.csv")
dec19 = pd.read_csv("../data/2019-Dec.csv")
jan20 = pd.read_csv("../data/2020-Jan.csv")
feb20 = pd.read_csv("../data/2020-Feb.csv")

In [9]:
print(oct19.shape)
print(nov19.shape)
print(dec19.shape)
print(jan20.shape)
print(feb20.shape)
print(oct19.shape[0]+nov19.shape[0]+dec19.shape[0]+jan20.shape[0]+feb20.shape[0])

(4102283, 9)
(4635837, 9)
(3533286, 9)
(4264752, 9)
(4156682, 9)
20692840


In [10]:
ecom_df = pd.concat([oct19, nov19, dec19, jan20, feb20], axis=0)
ecom_df.head()
print(ecom_df.shape)

(20692840, 9)


In [11]:
ecom_df = ecom_df[ecom_df.event_type != 'view']
ecom_df = ecom_df[ecom_df.event_type != 'cart']

# Data Cleaning

### NAs and how to deal with them

In [12]:
#8757117 nas is Brand
#4598 nas in user_session
#20339246 nas in category_code DROPPED

In [13]:
#NAs in Brand
ecom_df.brand.fillna(value="unknown", axis=0, inplace=True)

In [14]:
#NAs in user_session 
ecom_df.dropna(inplace=True)

In [15]:
ecom_df.category_code.isna().sum()

0

In [16]:
ecom_df.category_code.value_counts()

appliances.environment.vacuum             16225
stationery.cartrige                       15751
apparel.glove                             15683
furniture.bathroom.bath                    4086
furniture.living_room.cabinet              2046
accessories.bag                             754
accessories.cosmetic_bag                    500
appliances.personal.hair_cutter             276
appliances.personal.massager                212
appliances.environment.air_conditioner      104
furniture.living_room.chair                  26
Name: category_code, dtype: int64

In [17]:
#Drop this column, as we will use category code instead
ecom_df.drop("category_id", axis=1, inplace = True)

In [18]:
#Get rid of negative prices
ecom_df = ecom_df[ecom_df['price']>= 0]

# Feature Engineering

purchase per session, group session, total of price added up for each purchase in session

### Time FE

In [19]:
ecom_df['event_time'] = pd.to_datetime(ecom_df['event_time'])

In [20]:
ecom_df['month'] = pd.DatetimeIndex(ecom_df['event_time']).month
ecom_df['2020'] = pd.DatetimeIndex(ecom_df['event_time']).year
ecom_df['hour'] = pd.DatetimeIndex(ecom_df['event_time']).hour
ecom_df.head()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,month,year,hour
264,2019-10-01 00:07:58+00:00,remove_from_cart,5767493,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0
269,2019-10-01 00:08:03+00:00,remove_from_cart,5759489,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0
274,2019-10-01 00:08:08+00:00,remove_from_cart,5657894,stationery.cartrige,unknown,2.06,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0
276,2019-10-01 00:08:10+00:00,remove_from_cart,24336,stationery.cartrige,depilflax,3.02,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0
278,2019-10-01 00:08:18+00:00,remove_from_cart,5775822,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0


In [21]:
#convert year to string
ecom_df['2020'] = ecom_df['2020'].replace({2019: 0, 2020: 1

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,month,year,hour
264,2019-10-01 00:07:58+00:00,remove_from_cart,5767493,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0
269,2019-10-01 00:08:03+00:00,remove_from_cart,5759489,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0
274,2019-10-01 00:08:08+00:00,remove_from_cart,5657894,stationery.cartrige,unknown,2.06,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0
276,2019-10-01 00:08:10+00:00,remove_from_cart,24336,stationery.cartrige,depilflax,3.02,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0
278,2019-10-01 00:08:18+00:00,remove_from_cart,5775822,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,10,2019,0


In [22]:
ecom_df['day'] = ecom_df['event_time'].dt.dayofweek
ecom_df['day'] = ecom_df['day'].replace({0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'})

In [23]:
when_df = pd.DataFrame({'hour':range(1, 25)})
bins = [0,6,12,18,24]
labels = ['Night', 'Morning','Afternoon','Evening']
when_df['when'] = pd.cut(when_df['hour'], bins=bins, labels=labels, include_lowest=True)
#fix bellow
ecom_df = ecom_df.join(when_df['when'], on="hour")

In [24]:
ecom_df['month'] = ecom_df['month'].replace({1: 'January', 2: 'Febuary', 10: 'October', 11: 'November', 12: 'December'})

In [25]:
ecom_df.sample()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,month,year,hour,day,when
3840429,2020-01-28 22:27:20+00:00,remove_from_cart,5921391,apparel.glove,benovy,7.16,131716474,c1916cc9-8e3c-4e55-9f74-85899dacde04,January,2020,22,Tuesday,Evening


In [26]:
#ecom_df["current_session_clicks"] = ecom_df.groupby(['user_session'])['event_type'].cumsum()

In [27]:
#need to turn the features into numeric for bellow to work
#ecom_df["current_session_event_types"] = ecom_df.groupby(['user_session'])['event_type'].cumsum()

# Labeling

In [28]:
#encoder = LabelBinarizer()
#etype_labels = pd.DataFrame(encoder.fit_transform(ecom_df['event_type']), columns=encoder.classes_, index=ecom_df.index)
#etype_labels.head()

In [29]:
encoder = LabelBinarizer()
brand_labels = pd.DataFrame(encoder.fit_transform(ecom_df['brand']), columns=encoder.classes_, index=ecom_df.index)
brand_labels.head()

,babyliss,benovy,cutrin,depilflax,dewal,domix,emil,farmavita,farmona,gehwol,...,nitrimax,oniq,polarus,profepil,runail,shifei,shik,unknown,vosev,yoko
264,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
269,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
274,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
276,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
278,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
encoder = LabelBinarizer()
productid_labels = pd.DataFrame(encoder.fit_transform(ecom_df['product_id']), columns=encoder.classes_, index=ecom_df.index)
productid_labels.head()

,4923,4962,4965,4986,5395,6736,6737,6817,8098,8372,...,5926444,5926445,5926580,5926583,5926585,5926586,5928359,5929649,5929650,5929651
264,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
269,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
274,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
276,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
278,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
ohe = OneHotEncoder(sparse=False, drop='first')
catco_labels = pd.DataFrame(ohe.fit_transform(ecom_df[['category_code']]), index=ecom_df.index, columns=ohe.get_feature_names(ecom_df[["category_code"]].columns))
catco_labels.head()

,category_code_accessories.cosmetic_bag,category_code_apparel.glove,category_code_appliances.environment.air_conditioner,category_code_appliances.environment.vacuum,category_code_appliances.personal.hair_cutter,category_code_appliances.personal.massager,category_code_furniture.bathroom.bath,category_code_furniture.living_room.cabinet,category_code_furniture.living_room.chair,category_code_stationery.cartrige
264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
278,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [32]:
ohe = OneHotEncoder(sparse=False, drop='first')
day_labels = pd.DataFrame(ohe.fit_transform(ecom_df[['day']]), index=ecom_df.index, columns=ohe.get_feature_names(ecom_df[["day"]].columns))
day_labels.head()

,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday
264,0.0,0.0,0.0,0.0,1.0,0.0
269,0.0,0.0,0.0,0.0,1.0,0.0
274,0.0,0.0,0.0,0.0,1.0,0.0
276,0.0,0.0,0.0,0.0,1.0,0.0
278,0.0,0.0,0.0,0.0,1.0,0.0


In [33]:
ohe = OneHotEncoder(sparse=False, drop='first')
month_labels = pd.DataFrame(ohe.fit_transform(ecom_df[['month']]), index=ecom_df.index, columns=ohe.get_feature_names(ecom_df[["month"]].columns))
month_labels.head()

,month_Febuary,month_January,month_November,month_October
264,0.0,0.0,0.0,1.0
269,0.0,0.0,0.0,1.0
274,0.0,0.0,0.0,1.0
276,0.0,0.0,0.0,1.0
278,0.0,0.0,0.0,1.0


In [34]:
ohe = OneHotEncoder(sparse=False, drop='first')
when_labels = pd.DataFrame(ohe.fit_transform(ecom_df[['when']]), index=ecom_df.index, columns=ohe.get_feature_names(ecom_df[["when"]].columns))
when_labels.head()

,when_Evening,when_Morning,when_Night
264,0.0,0.0,1.0
269,0.0,0.0,1.0
274,0.0,0.0,1.0
276,0.0,0.0,1.0
278,0.0,0.0,1.0


In [36]:
ecom_df.head()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,month,year,hour,day,when
264,2019-10-01 00:07:58+00:00,remove_from_cart,5767493,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,October,2019,0,Tuesday,Night
269,2019-10-01 00:08:03+00:00,remove_from_cart,5759489,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,October,2019,0,Tuesday,Night
274,2019-10-01 00:08:08+00:00,remove_from_cart,5657894,stationery.cartrige,unknown,2.06,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,October,2019,0,Tuesday,Night
276,2019-10-01 00:08:10+00:00,remove_from_cart,24336,stationery.cartrige,depilflax,3.02,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,October,2019,0,Tuesday,Night
278,2019-10-01 00:08:18+00:00,remove_from_cart,5775822,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,October,2019,0,Tuesday,Night


In [37]:
#Join our Labeled Columns to our main DataFrame
#ecom_df = ecom_df.join(etype_labels)
ecom_df = ecom_df.join(brand_labels)
ecom_df = ecom_df.join(productid_labels)
ecom_df = ecom_df.join(catco_labels)
ecom_df = ecom_df.join(day_labels)
ecom_df = ecom_df.join(month_labels)
ecom_df = ecom_df.join(when_labels)                        
ecom_df.head()

,event_time,event_type,product_id,category_code,brand,price,user_id,user_session,month,year,...,day_Tuesday,day_Wednesday,month_Febuary,month_January,month_November,month_October,when_Evening,when_Morning,when_Night,year_2020
66,2019-11-01 00:04:33+00:00,purchase,5766980,stationery.cartrige,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,November,2019,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
67,2019-11-01 00:04:33+00:00,purchase,5767494,stationery.cartrige,italwax,2.14,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,November,2019,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
68,2019-11-01 00:04:33+00:00,purchase,5775813,stationery.cartrige,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,November,2019,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
69,2019-11-01 00:04:33+00:00,purchase,5775814,stationery.cartrige,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f,November,2019,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
264,2019-10-01 00:07:58+00:00,remove_from_cart,5767493,stationery.cartrige,italwax,2.14,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,October,2019,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


# Preprocessing Data

### Train Test Split

In [67]:
X = ecom_df.drop(["month", "year", "hour", "day", "when", "user_id", "product_id", "category_code", "event_time", "event_type", "brand", "user_session"], axis=1)
y = ecom_df.event_type

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Standard Scalar

In [69]:
ss = StandardScaler()
X_train_ss = pd.DataFrame(ss.fit_transform(X_train), index=X_train.index, columns=X_train.columns)  

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [70]:
ss = StandardScaler()
X_test_ss = pd.DataFrame(ss.fit_transform(X_test), index=X_test.index, columns=X_test.columns)

AttributeError: 'numpy.ndarray' object has no attribute 'index'

### SMOTE

In [42]:
sm = SMOTE(random_state=42)
X_tr_sm, y_tr_sm = sm.fit_resample(X_train_ss, y_train)

In [74]:
# summarize class distribution
counter = Counter(y)
print(counter)
# define the undersampling method
undersample = NearMiss(version=3, n_neighbors=3)
# transform the dataset
X, y = undersample.fit_resample(X, y)
# summarize the new class distribution
counter = Counter(y)
print(counter)

Counter({'purchase': 38704, 'remove_from_cart': 28081})


/opt/anaconda3/lib/python3.8/site-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:177: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(


Counter({'remove_from_cart': 28081, 'purchase': 20654})


# Models

### LGBMClassifier

In [43]:
import lightgbm as lgb

In [47]:
# Instantiate XGBClassifier
lgbm = lgb.LGBMClassifier()

# Fit XGBClassifier
lgbm.fit(X_train_ss,  y_train)

# Predict on training and test sets
training_preds_lgbm = lgbm.predict(X_train_ss)
test_preds_lgbm = lgbm.predict(X_test_ss)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_lgbm)
print(first_results)

                  precision    recall  f1-score   support

        purchase       0.76      0.30      0.43      7792
remove_from_cart       0.77      0.96      0.86     19426

        accuracy                           0.77     27218
       macro avg       0.77      0.63      0.64     27218
    weighted avg       0.77      0.77      0.73     27218



In [83]:
# Instantiate XGBClassifier
lgbm = lgb.LGBMClassifier()

# Fit XGBClassifier
lgbm.fit(X_train,  y_train)

# Predict on training and test sets
training_preds_lgbm = lgbm.predict(X_train)
test_preds_lgbm = lgbm.predict(X_test)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_lgbm)
print(first_results)

                  precision    recall  f1-score   support

        purchase       0.76      0.23      0.35      4094
remove_from_cart       0.63      0.95      0.76      5653

        accuracy                           0.65      9747
       macro avg       0.69      0.59      0.55      9747
    weighted avg       0.68      0.65      0.59      9747



In [49]:
# Instantiate XGBClassifier
lgbm = lgb.LGBMClassifier()

# Fit XGBClassifier
lgbm.fit(X_tr_sm,  y_tr_sm)

# Predict on training and test sets
training_preds_lgbm = lgbm.predict(X_tr_sm)
test_preds_lgbm = lgbm.predict(X_test_ss)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_lgbm)
print(first_results)

                  precision    recall  f1-score   support

        purchase       0.29      1.00      0.45      7792
remove_from_cart       1.00      0.00      0.00     19426

        accuracy                           0.29     27218
       macro avg       0.64      0.50      0.22     27218
    weighted avg       0.80      0.29      0.13     27218



### RandomForest

In [84]:
rf = RandomForestClassifier()

rf.fit(X_train, y_train)

# Predict on training and test sets
training_preds_rf = rf.predict(X_train)
test_preds_rf = rf.predict(X_test)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_rf)
print(first_results)

                  precision    recall  f1-score   support

        purchase       0.52      0.46      0.49      4094
remove_from_cart       0.64      0.69      0.66      5653

        accuracy                           0.59      9747
       macro avg       0.58      0.57      0.57      9747
    weighted avg       0.59      0.59      0.59      9747



### DecisionTreeClassifier

In [85]:
tree_clf = DecisionTreeClassifier() 

tree_clf.fit(X_train, y_train)

# Predict on training and test sets
training_preds_tclf = tree_clf.predict(X_train)
test_preds_tclf = tree_clf.predict(X_test)

# Printing the Classification Report to see how our model performed
first_results=classification_report(y_test, test_preds_tclf)
print(first_results)

                  precision    recall  f1-score   support

        purchase       0.48      0.55      0.51      4094
remove_from_cart       0.63      0.56      0.59      5653

        accuracy                           0.56      9747
       macro avg       0.55      0.55      0.55      9747
    weighted avg       0.57      0.56      0.56      9747



# Visuals

### Graph 1

In [30]:
user_df = ecom_df[['user_id','event_type','price']]

In [33]:
graph_df = user_df[['user_id','price']]
graph_df = graph_df.groupby(['user_id'])['price'].sum().reset_index()
graph_df = graph_df.sort_values(by=['price'],ascending=False)
graph_df = graph_df.rename(columns={'price':'spent'})

In [37]:
price_list = graph_df['spent'].to_list()
plt.hist(x= price_list,bins=100,log=True, color='green')
plt.xlabel('Spent amount')
plt.ylabel('Customer Count')
plt.title('Customer Spending')
plt.savefig("Customer_Spend")

### Graph 2

In [47]:
# Daily visitors number
date = ecom_df.loc[:,['event_time','user_id']]
visitor_by_date = date.groupby(['event_time'])['user_id'].nunique()
visitor_by_date

event_time
2019-10-01 00:00:53+00:00    1
2019-10-01 00:07:58+00:00    1
2019-10-01 00:08:03+00:00    1
2019-10-01 00:08:08+00:00    1
2019-10-01 00:08:10+00:00    1
                            ..
2020-02-29 23:47:54+00:00    1
2020-02-29 23:51:55+00:00    1
2020-02-29 23:54:28+00:00    1
2020-02-29 23:54:43+00:00    1
2020-02-29 23:54:44+00:00    1
Name: user_id, Length: 330514, dtype: int64

### Graph 3

In [66]:
# scatter plot of examples by class label
for label, _ in counter.items():
    row_ix = where(y == label)[0]
    plt.scatter(X[row_ix, 0], X[row_ix, 1], label=str(label))
plt.legend()
plt.savefig("class_imbalance_before")